In [3]:
! pip install scikit-learn
! pip install keras_tuner
! pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 936.1 kB/s eta 0:00:00a 0:00:01


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
iris = datasets.load_iris()
# Use the first 4 variables to predict the species.
X, y = iris.data[:, :4], iris.target

# Split both independent and dependent variables in half
# for cross-validation
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)


# Keras_tuner

In [14]:
import keras_tuner
from tensorflow import keras

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
    hp.Choice('units', [8, 16, 32]), # choice of param values
        activation='relu'
    ))
    model.add(keras.layers.Dense(1, activation='relu'))
    model.compile(loss='mse')
    return model

# Random searching for best hyperparameter
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5
)
tuner.search(
    x_train, y_train,
    epochs=5,
    validation_data=(x_test, y_test)
)
best_model = tuner.get_best_models()[0]

Trial 3 Complete [00h 00m 01s]
val_loss: 1.2333333492279053

Best val_loss So Far: 1.2333333492279053
Total elapsed time: 00h 00m 03s
INFO:tensorflow:Oracle triggered exit


In [18]:
best_model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'units': 32,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 1,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}}]}

# Sklearn's grid search

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

grid_params = {
    "n_neighbors": [3, 5, 7, 10, 15, 20, 25, 35],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan"]
}

gs = GridSearchCV(
    KNeighborsClassifier(),
    grid_params,
    cv=10,
    verbose=1
)

gs_results = gs.fit(X, y)

# best param and scores for your model can be obtained as follows
print(
    "Best k: ",
    gs_results.best_estimator_.get_params()["n_neighbors"]
)
print(gs_results.best_score_, gs_results.best_params_)

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Best k:  15
0.9800000000000001 {'metric': 'euclidean', 'n_neighbors': 15, 'weights': 'distance'}


In [20]:
from sklearn.model_selection import RandomizedSearchCV
grid_params = {
    "n_neighbors": [3, 5, 7, 10, 15, 20, 25, 35],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan"]
}

rs = RandomizedSearchCV(
    KNeighborsClassifier(),
    param_distributions=grid_params,
    n_iter=10
)

rs_results = rs.fit(X, y)

# best param and scores for your model can be obtained as follows
print(
    "Best k: ",
    rs_results.best_estimator_.get_params()["n_neighbors"]
)
print(rs_results.best_score_, rs_results.best_params_)

Best k:  10
0.9733333333333334 {'weights': 'distance', 'n_neighbors': 10, 'metric': 'manhattan'}
